In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train=pd.read_csv("../input/spaceship-titanic/train.csv")
test=pd.read_csv("../input/spaceship-titanic/test.csv")
ss=pd.read_csv("../input/spaceship-titanic/sample_submission.csv")

In [4]:
ytrain=train.Transported
ytrain=ytrain.astype(int)
ytrain

passid=test.PassengerId

train.drop("Transported",axis=1,inplace=True)
dataset=pd.concat([train,test],axis=0)

dataset=dataset.drop(['Name','Cabin'],axis=1)
col=['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Age']
dataset[col]=dataset[col].fillna(dataset[col].median())

dataset=dataset.fillna({'HomePlanet':'Earth','CryoSleep':'False','Destination':'TRAPPIST-1e','VIP':'False'})

dataset= dataset.astype({"CryoSleep":'bool', "VIP":'bool'})
bool1= {True:1,False:0}
planet= {"Europa":1,'Earth':2,'Mars':3}
dest= {"TRAPPIST-1e":1,"PSO J318.5-22":2,"55 Cancri e":3}
dataset['HomePlanet'] = dataset['HomePlanet'].map(planet)
dataset['CryoSleep'] = dataset['CryoSleep'].map(bool1)
dataset['Destination'] =dataset['Destination'].map(dest)
dataset['VIP'] =dataset['VIP'].map(bool1)

dataset.drop('PassengerId',axis=1,inplace=True)
train_set=dataset.iloc[:8693,]
train_set.info()
test_set=dataset.iloc[8693:,]

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(train_set,ytrain,train_size=0.8,random_state=1)
from sklearn.preprocessing import StandardScaler
x_train=StandardScaler().fit_transform(x_train)
x_test=StandardScaler().fit_transform(x_test)
test_set=StandardScaler().fit_transform(test_set)


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model=keras.Sequential(
[
        keras.Input(shape=(10,)),
        #layers.Flatten(input_shape=(10,)),
        layers.Dense(100,activation='relu',name="layer1"),
        layers.Dense(50,activation='relu'),
        layers.Dense(1,activation='sigmoid'),
    ]
)

model.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)
model.summary()
model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test))

y_pred=model.predict(test_set)
y_pred.shape


<class 'pandas.core.frame.DataFrame'>
Int64Index: 8693 entries, 0 to 8692
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8693 non-null   int64  
 1   CryoSleep     8693 non-null   int64  
 2   Destination   8693 non-null   int64  
 3   Age           8693 non-null   float64
 4   VIP           8693 non-null   int64  
 5   RoomService   8693 non-null   float64
 6   FoodCourt     8693 non-null   float64
 7   ShoppingMall  8693 non-null   float64
 8   Spa           8693 non-null   float64
 9   VRDeck        8693 non-null   float64
dtypes: float64(6), int64(4)
memory usage: 747.1 KB


2022-07-31 06:37:41.316708: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-07-31 06:37:41.469022: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               (None, 100)               1100      
_________________________________________________________________
dense (Dense)                (None, 50)                5050      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 6,201
Trainable params: 6,201
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
218/218 [==============================] - 1s 3ms/step - loss: 0.4977 - accuracy: 0.7610 - val_loss: 0.4451 - val_accuracy: 0.8051
Epoch 2/100
218/218 [==============================] - 0s 2ms/step - loss: 0.4512 - accuracy: 0.7859 - val_loss: 0.4326 - val_accuracy: 0.8068
Epoch 3/100
218/218 [==============================] - 0s 2ms/step - loss: 0.4419 - accuracy: 0.78

(4277, 1)

In [5]:
y_pred1=[]
for i in y_pred:
    if i>0.5:
        y_pred1.append(1)
    else:
        y_pred1.append(0)
        
y_pred1

transportedpredframe=pd.DataFrame(y_pred1, columns=['Transported'])
transportedpredframe['Transported']=transportedpredframe['Transported'].replace(1, "True")
transportedpredframe['Transported']=transportedpredframe['Transported'].replace(0, "False")
transportedpredframe

,Transported
0,True
1,False
2,True
3,True
4,True
...,...
4272,True
4273,False
4274,True
4275,True


In [6]:
pass_id=pd.DataFrame(passid, columns=['PassengerId'])
submission=pd.concat([pass_id,transportedpredframe],axis=1)
submission.to_csv("submission.csv", index=False)